In [1]:
import cv2
import numpy as np
from pygame import mixer
from tensorflow.keras.models import load_model

# Initialize pygame mixer
mixer.init()
sound = mixer.Sound(r'D:\Davies Ningthoujam\My project\Ml Project\Driver drowsiness Detection\alarm.wav')


# Load the cascades and model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'D:\Davies Ningthoujam\My project\Ml Project\Driver drowsiness Detection\models')

# Start video capture
cap = cv2.VideoCapture(0)
Score = 0

while True:
    ret, frame = cap.read()
 
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)  # Apply histogram equalization
    
    # Calculate average brightness and adjust if necessary
    brightness = np.mean(gray)
    if brightness > 180:
        frame = cv2.convertScaleAbs(frame, alpha=0.5, beta=0)
    elif brightness < 50:
        frame = cv2.convertScaleAbs(frame, alpha=1.5, beta=20)
        
    background_color = (0, 0, 0)  # Black background color for the rectangle
    text_color = (255, 255, 255)

    # Detect faces and eyes
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 3)
        face_roi = gray[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.2, minNeighbors=3)

        for (ex, ey, ew, eh) in eyes:
            ex, ey = x + ex, y + ey  # Adjust coordinates
            eye = frame[ey:ey + eh, ex:ex + ew]
            eye = cv2.resize(eye, (80, 80)) / 255.0
            eye = np.expand_dims(eye.reshape(80, 80, 3), axis=0)
            prediction = model.predict(eye)
            
            if prediction[0][0] > 0.30:
                cv2.rectangle(frame, (width - 160, height - 40), (width - 10, height), background_color, -1)
                cv2.putText(frame,'C',(width - 150,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
                cv2.putText(frame,'Score'+str(Score),(width - 100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
                Score += 1
                if Score > 20:
                    try:
                        sound.play()
                        pass
                    except:
                        pass
            elif prediction[0][1] > 0.65:
                cv2.rectangle(frame, (0, height - 40), (200, height), background_color, -1)    
                        
                cv2.putText(frame,'O',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)      
                cv2.putText(frame,'Score'+str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
                Score = Score-2
                if (Score<0):
                    Score=0
            

    cv2.imshow('frame', frame)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.15)
Hello from the pygame community. https://www.pygame.org/contribute.html
1/1 [==============================] - 0s 52ms/step


KeyboardInterrupt: 